In [1]:
import pandas as pd
#pd.set_option("display.max_colwidth", -1)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import re
import string
import math
from collections import Counter
import spacy
import re
import plotly.express as px
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import sys
import warnings
from wordcloud import WordCloud,STOPWORDS
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

ModuleNotFoundError: No module named 'plotly'

In [ ]:
df = pd.read_csv(r'C:\Users\nitis\OneDrive\Desktop\thesis\Nitish_datasets\Nitish_datasets\df_sentences_with_cite.csv')
df.head(1)

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92610 entries, 0 to 92609
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   article_id             92610 non-null  int64 
 1   sentence_id            92610 non-null  object
 2   para_id                92610 non-null  object
 3   sentence               92608 non-null  object
 4   section_nr             92610 non-null  int64 
 5   last_section_title     70468 non-null  object
 6   last_subsection_title  16496 non-null  object
 7   sentence_type          92610 non-null  object
 8   number_of_citations    92610 non-null  int64 
 9   sentence_with_cite     92608 non-null  object
 10  has_citation           92610 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 7.8+ MB


In [173]:
data = df.dropna(subset=['sentence','last_section_title'])
data = data.drop(data[(data['sentence_type'].str.contains("HEADER|ANNEX"))  ].index)
data['sentence'] = data['sentence'].str.lower()
data['last_section_title'] = data['last_section_title'].str.lower()

In [174]:
data.head()

,article_id,sentence_id,para_id,sentence,section_nr,last_section_title,last_subsection_title,sentence_type,number_of_citations,sentence_with_cite,has_citation
3,1,1_21_38,1_1,social media and online communities provide organizations with new opportunities to support their businessrelated functions .,0,abstract,NaN,ABSTRACT,0,Social media and online communities provide organizations with new opportunities to support their businessrelated functions .,0
4,1,1_38_55,1_1,"despite their various benefits , social media technologies present two important challenges for sense - making .",0,abstract,NaN,ABSTRACT,0,"Despite their various benefits , social media technologies present two important challenges for sense - making .",0
5,1,1_55_73,1_1,"first , online discourse is plagued by incoherent , intertwined conversations that are often difficult to comprehend .",0,abstract,NaN,ABSTRACT,0,"First , online discourse is plagued by incoherent , intertwined conversations that are often difficult to comprehend .",0
6,1,1_73_104,1_1,"moreover , organizations are increasingly interested in understanding social media participants ' actions and intentions ; however , existing text analytics tools mostly focus on the semantic dimension of language .",0,abstract,NaN,ABSTRACT,0,"Moreover , organizations are increasingly interested in understanding social media participants ' actions and intentions ; however , existing text analytics tools mostly focus on the semantic dimension of language .",0
7,1,1_104_132,1_1,,0,abstract,NaN,ABSTRACT,0,"The language - action perspective ( language-action perspective ) emphasizes pragmatics ; not what people say but , rather , what they do with language .",0


In [175]:
aim_key = ["aim ","goal ","purpose ","we describe","we analyse","we analyze","we describe","we demonstrate"]
pattern_aim = '|'.join([f'(?i){key}' for key in aim_key])

motivation_key = ["motivation","problem","objective","limitation"]
pattern_motivation = '|'.join([f'(?i){key}' for key in motivation_key])

defination_key = ["is defined","defination","defined as","we define", "define"]
pattern_defination = '|'.join([f'(?i){key}' for key in defination_key])


hypothesis_result_key = ["support","decline","reject","hypothesis"]
pattern_hypothesis = '|'.join([f'(?i){key}' for key in hypothesis_result_key])


futurework_key = ["future","further research"]
pattern_futurework = '|'.join([f'(?i){key}' for key in futurework_key])

In [176]:
# RULE CLASS 1

#defination
conditions7 = [(data['sentence'].str.contains(pattern_defination))]

#limitation
conditions12 = [(data['last_section_title'].str.contains("limitation|conclusion"))  
                & (data['sentence'].str.contains("limitation"))]

#future work
conditions13 = [(data['last_section_title'].str.contains("limitation|conclusion|future work|contribution"))  
                & (data['sentence'].str.contains(pattern_futurework))
               ]

#contribution
conditions11 = [(data['last_section_title'].str.contains("(?i)abstract|introduction|conclusion|contribution"))  
                & (data['sentence'].str.contains("(?i)result|contribution|we propose")) 
               ]


#hypothesis result
conditions9 = [(data['last_section_title'].str.contains("(?i)result|hypothesis")) 
               & (data['section_nr'] > 1) 
               & (data['sentence'].str.contains(pattern_hypothesis))]

#hypothesis
conditions8 = [(data['last_section_title'].str.contains("(?i)hyp"))
              & (~data['sentence'].str.contains(pattern_hypothesis))
              ]


#method
conditions6 = [(data['last_section_title'].str.contains("method|methodology|methodological")) 
               & (data['section_nr'] > 1) 
               & (data['sentence'].str.contains("we "))
               & (data['has_citation'] != 1) 
               ]

#Research Question
conditions4 = [(data['last_section_title'].str.contains("(?i)Research question|research objective")) ]

#Motivation
conditions3 = [(data['section_nr'] == 1) 
               &  (data['sentence'].str.contains(pattern_motivation))  
               ]


#AIM
conditions2 =  [(data['section_nr'] == 1) 
               & (data['sentence'].str.contains(pattern_aim)) 
               ]




#Emperical result
conditions10 = [(data['last_section_title'].str.contains("result|analysis|emperical")) 
                & (data['section_nr'] > 1) 
                & (~data['last_section_title'].str.contains("(?i)hypo"))
                & (data['has_citation'] != 1) 
               
               ]


#Relation Literature 
conditions5 = [(data['has_citation'] == 1) 
               & (data['sentence'].str.contains("(?i)we |our ")) 
               ]



#Relation Literature 
conditions14=[(data['section_nr'] == 0)
               & (data['last_section_title'].str.contains("(?i)abstract"))  
               ]






choices = [1]

data['aim'] = np.select(conditions2, choices, default=0)
data['motivation'] = np.select(conditions3, choices, default=0)
data['research_question'] = np.select(conditions4, choices, default=0)
data['relation_literature'] = np.select(conditions5, choices, default=0)
data['method'] = np.select(conditions6, choices, default=0)
data['Defination'] = np.select(conditions7, choices, default=0)
data['hypothesis'] = np.select(conditions8, choices, default=0)
data['hypothesis result'] = np.select(conditions9, choices, default=0)
data['Emperical result'] = np.select(conditions10, choices, default=0)
data['contribution'] = np.select(conditions11, choices, default=0)
data['limitation'] = np.select(conditions12, choices, default=0)
data['future work'] = np.select(conditions13, choices, default=0)
data['Abstract'] = np.select(conditions14, choices, default=0)

data.head()

,article_id,sentence_id,para_id,sentence,section_nr,last_section_title,last_subsection_title,sentence_type,number_of_citations,sentence_with_cite,has_citation,aim,motivation,research_question,relation_literature,method,Defination,hypothesis,hypothesis result,Emperical result,contribution,limitation,future work,Abstract
3,1,1_21_38,1_1,social media and online communities provide organizations with new opportunities to support their businessrelated functions .,0,abstract,NaN,ABSTRACT,0,Social media and online communities provide organizations with new opportunities to support their businessrelated functions .,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,1_38_55,1_1,"despite their various benefits , social media technologies present two important challenges for sense - making .",0,abstract,NaN,ABSTRACT,0,"Despite their various benefits , social media technologies present two important challenges for sense - making .",0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,1,1_55_73,1_1,"first , online discourse is plagued by incoherent , intertwined conversations that are often difficult to comprehend .",0,abstract,NaN,ABSTRACT,0,"First , online discourse is plagued by incoherent , intertwined conversations that are often difficult to comprehend .",0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,1,1_73_104,1_1,"moreover , organizations are increasingly interested in understanding social media participants ' actions and intentions ; however , existing text analytics tools mostly focus on the semantic dimension of language .",0,abstract,NaN,ABSTRACT,0,"Moreover , organizations are increasingly interested in understanding social media participants ' actions and intentions ; however , existing text analytics tools mostly focus on the semantic dimension of language .",0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,1,1_104_132,1_1,,0,abstract,NaN,ABSTRACT,0,"The language - action perspective ( language-action perspective ) emphasizes pragmatics ; not what people say but , rather , what they do with language .",0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [177]:
data.iloc[:, 10:] = data[data.columns[10:]].replace(0, np.nan)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63335 entries, 3 to 92120
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   article_id             63335 non-null  int64  
 1   sentence_id            63335 non-null  object 
 2   para_id                63335 non-null  object 
 3   sentence               63335 non-null  object 
 4   section_nr             63335 non-null  int64  
 5   last_section_title     63335 non-null  object 
 6   last_subsection_title  14568 non-null  object 
 7   sentence_type          63335 non-null  object 
 8   number_of_citations    63335 non-null  int64  
 9   sentence_with_cite     63335 non-null  object 
 10  has_citation           10232 non-null  float64
 11  aim                    94 non-null     float64
 12  motivation             184 non-null    float64
 13  research_question      68 non-null     float64
 14  relation_literature    1738 non-null   float64
 15  me

In [178]:
#RULE CLASS 2
conditions_second = [
    (data['Defination'] == 1),
    (data['limitation'] == 1),
    (data['future work'] == 1),
    (data['contribution'] == 1),
    (data['hypothesis result'] == 1),
    (data['hypothesis'] == 1),
    (data['method'] == 1),
    (data['research_question'] == 1),
    (data['motivation'] == 1),
    (data['aim'] == 1),
    (data['Emperical result'] == 1),
    (data['relation_literature'] == 1),
    (data['Abstract'] == 1),
    (data['Abstract'] == 0)
    
]

choices3 = ['Defination','Limitation','Future Work','Contribution','Hypothesis Result','Hypothesis','Method','Research Question','Motivation','Aim','Emperical Result','Relation to literature' ,'Abstract', np.nan]
data['Labels'] = np.select(conditions_second, choices3, default=np.nan)
data

,article_id,sentence_id,para_id,sentence,section_nr,last_section_title,last_subsection_title,sentence_type,number_of_citations,sentence_with_cite,has_citation,aim,motivation,research_question,relation_literature,method,Defination,hypothesis,hypothesis result,Emperical result,contribution,limitation,future work,Abstract,Labels
3,1,1_21_38,1_1,social media and online communities provide organizations with new opportunities to support their businessrelated functions .,0,abstract,NaN,ABSTRACT,0,Social media and online communities provide organizations with new opportunities to support their businessrelated functions .,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Abstract
4,1,1_38_55,1_1,"despite their various benefits , social media technologies present two important challenges for sense - making .",0,abstract,NaN,ABSTRACT,0,"Despite their various benefits , social media technologies present two important challenges for sense - making .",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Abstract
5,1,1_55_73,1_1,"first , online discourse is plagued by incoherent , intertwined conversations that are often difficult to comprehend .",0,abstract,NaN,ABSTRACT,0,"First , online discourse is plagued by incoherent , intertwined conversations that are often difficult to comprehend .",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Abstract
6,1,1_73_104,1_1,"moreover , organizations are increasingly interested in understanding social media participants ' actions and intentions ; however , existing text analytics tools mostly focus on the semantic dimension of language .",0,abstract,NaN,ABSTRACT,0,"Moreover , organizations are increasingly interested in understanding social media participants ' actions and intentions ; however , existing text analytics tools mostly focus on the semantic dimension of language .",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Abstract
7,1,1_104_132,1_1,,0,abstract,NaN,ABSTRACT,0,"The language - action perspective ( language-action perspective ) emphasizes pragmatics ; not what people say but , rather , what they do with language .",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Abstract
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92116,88,88_18307_18317,88_159,i remained confident that i could overcome the failure .,87,limitations and future topics,NaN,PARAGRAPH,0,I remained confident that I could overcome the failure .,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
92117,88,88_18318_18330,88_159,i was not anymore confident that i could overcome the failure .,87,limitations and future topics,NaN,PARAGRAPH,0,I was not anymore confident that I could overcome the failure .,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
92118,88,88_18333_18362,88_160,"[ if answered "" i did not do ( or try to do ) any further activities "" or reported that the incident was no longer relevant : ]",87,limitations and future topics,NaN,PARAGRAPH,0,"[ If answered "" I did not do ( or try to do ) any further activities "" or reported that the incident was no longer relevant : ]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
92119,88,88_18365_18378,88_161,please briefly describe your final thoughts and feelings about the failure experience .,87,limitations and future topics,NaN,PARAGRAPH,0,Please briefly describe your final thoughts and feelings about the failure experience .,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan


In [179]:
 data[(data["Labels"] == 'nan')].head(20)

,article_id,sentence_id,para_id,sentence,section_nr,last_section_title,last_subsection_title,sentence_type,number_of_citations,sentence_with_cite,has_citation,aim,motivation,research_question,relation_literature,method,Defination,hypothesis,hypothesis result,Emperical result,contribution,limitation,future work,Abstract,Labels
1981,1,1_312_332,1_3,the rapid growth of social media and online communities has dramatically changed the manner in which communication takes place .,1,introduction,NaN,PARAGRAPH,0,The rapid growth of social media and online communities has dramatically changed the manner in which communication takes place .,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1983,1,1_355_409,1_3,"according to a mckinsey quarterly report , 50 variance explained of the more than 1,700 organizations surveyed are using social networking , 41 variance explained are using blogs , 25 variance explained are using wikis and 23 variance explained are using microblogs ( bughin and chui 2010 ) .",1,introduction,NaN,PARAGRAPH,1,"According to a McKinsey Quarterly report , 50 Variance Explained of the more than 1,700 organizations surveyed are using social networking , 41 Variance Explained are using blogs , 25 Variance Explained are using wikis and 23 Variance Explained are using microblogs CITE_bughin_j_the_rise_of_the_networked_enterprise_web_20_finds_its_payday_2010_mckinsey_quarterly .",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1985,1,1_435_459,1_3,"web 2.0 technologies are being leveraged for internal purposes , customer - related purposes , and to work with external suppliers and partners .",1,introduction,NaN,PARAGRAPH,0,"Web 2.0 technologies are being leveraged for internal purposes , customer - related purposes , and to work with external suppliers and partners .",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1986,1,1_459_515,1_3,"organizations are deriving considerable benefits from their use , including increased speed of access to knowledge , enhanced identification of experts , increased number of successful innovations , and reduced communication and operational costs ( bughin and chui 2010 ; chau and xu 2012 ) .",1,introduction,NaN,PARAGRAPH,2,"Organizations are deriving considerable benefits from their use , including increased speed of access to knowledge , enhanced identification of experts , increased number of successful innovations , and reduced communication and operational costs CITE_bughin_j_the_rise_of_the_networked_enterprise_web_20_finds_its_payday_2010_mckinsey_quarterly CITE_chau_m_business_intelligence_in_blogs_understanding_consumer_interactions_and_communities_2012_mis_quarterly .",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1987,1,1_518_556,1_4,sense - making is an information - processing task that serves as a critical prerequisite for decision - making ( russell et al . 1993 ; weick et al . 1995 ) .,1,introduction,NaN,PARAGRAPH,1,Sense - making is an information - processing task that serves as a critical prerequisite for decision - making CITE_russell_d_the_cost_structure_of_sensemaking_1993_book Weick et al . 1995 ) .,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1989,1,1_580_619,1_5,"communication modes such as chat rooms , newsgroups , forums , blogs , social networking discussions , and microblogs are highly susceptible to intertwined conversations and incoherence ( honeycutt and herring 2009 ) .",1,introduction,NaN,PARAGRAPH,1,"Communication modes such as chat rooms , newsgroups , forums , blogs , social networking discussions , and microblogs are highly susceptible to intertwined conversations and incoherence CITE_honeycutt_c_beyond_microblogging_conversation_and_collaboration_in_twitter_2009_hicss .",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1990,1,1_619_680,1_5,"in group discussion , these issues make it difficult for analysts and supporting technologies to determine the correct message - conversation aff

In [180]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63335 entries, 3 to 92120
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   article_id             63335 non-null  int64  
 1   sentence_id            63335 non-null  object 
 2   para_id                63335 non-null  object 
 3   sentence               63335 non-null  object 
 4   section_nr             63335 non-null  int64  
 5   last_section_title     63335 non-null  object 
 6   last_subsection_title  14568 non-null  object 
 7   sentence_type          63335 non-null  object 
 8   number_of_citations    63335 non-null  int64  
 9   sentence_with_cite     63335 non-null  object 
 10  has_citation           10232 non-null  float64
 11  aim                    94 non-null     float64
 12  motivation             184 non-null    float64
 13  research_question      68 non-null     float64
 14  relation_literature    1738 non-null   float64
 15  me

In [181]:
data.groupby(['Labels']).count()

,article_id,sentence_id,para_id,sentence,section_nr,last_section_title,last_subsection_title,sentence_type,number_of_citations,sentence_with_cite,has_citation,aim,motivation,research_question,relation_literature,method,Defination,hypothesis,hypothesis result,Emperical result,contribution,limitation,future work,Abstract
Labels,,,,,,,,,,,,,,,,,,,,,,,,
Abstract,1595,1595,1595,1595,1595,1595,0,1595,1595,1595,172,0,0,0,0,0,0,0,0,0,0,0,0,1595
Aim,84,84,84,84,84,84,7,84,84,84,25,84,0,0,6,0,0,0,0,0,0,0,0,0
Contribution,525,525,525,525,525,525,23,525,525,525,76,3,10,2,16,2,0,0,0,8,525,0,0,135
Defination,611,611,611,611,611,611,139,611,611,611,162,2,0,1,37,13,611,6,0,43,4,0,0,6
Emperical Result,3364,3364,3364,3364,3364,3364,1122,3364,3364,3364,0,0,0,0,0,0,0,0,0,3364,0,0,0,0
Future Work,237,237,237,237,237,237,24,237,237,237,24,0,0,2,8,0,0,0,0,5,17,0,237,0
Hypothesis,991,991,991,991,991,991,423,991,991,991,281,0,0,0,24,0,0,991,0,0,0,0,0,0
Hypothesis Result,162,162,162,162,162,162,40,162,162,162,9,0,0,0,2,0,0,0,162,101,0,0,0,0
Limitation,92,92,92,92,92,92,6,92,92,92,7,0,0,0,5,0,0,0,0,1,2,92,23,0


In [197]:
Labelled_data = data.drop(data.columns[11:24], axis=1 )
Labelled_data = Labelled_data.drop(Labelled_data[(Labelled_data['Labels'].str.contains("nan")) ].index)
Labelled_data = Labelled_data.reset_index(drop=True)

In [198]:
Labelled_data

,article_id,sentence_id,para_id,sentence,section_nr,last_section_title,last_subsection_title,sentence_type,number_of_citations,sentence_with_cite,has_citation,Labels
0,1,1_21_38,1_1,social media and online communities provide organizations with new opportunities to support their businessrelated functions .,0,abstract,NaN,ABSTRACT,0,Social media and online communities provide organizations with new opportunities to support their businessrelated functions .,NaN,Abstract
1,1,1_38_55,1_1,"despite their various benefits , social media technologies present two important challenges for sense - making .",0,abstract,NaN,ABSTRACT,0,"Despite their various benefits , social media technologies present two important challenges for sense - making .",NaN,Abstract
2,1,1_55_73,1_1,"first , online discourse is plagued by incoherent , intertwined conversations that are often difficult to comprehend .",0,abstract,NaN,ABSTRACT,0,"First , online discourse is plagued by incoherent , intertwined conversations that are often difficult to comprehend .",NaN,Abstract
3,1,1_73_104,1_1,"moreover , organizations are increasingly interested in understanding social media participants ' actions and intentions ; however , existing text analytics tools mostly focus on the semantic dimension of language .",0,abstract,NaN,ABSTRACT,0,"Moreover , organizations are increasingly interested in understanding social media participants ' actions and intentions ; however , existing text analytics tools mostly focus on the semantic dimension of language .",NaN,Abstract
4,1,1_104_132,1_1,,0,abstract,NaN,ABSTRACT,0,"The language - action perspective ( language-action perspective ) emphasizes pragmatics ; not what people say but , rather , what they do with language .",NaN,Abstract
...,...,...,...,...,...,...,...,...,...,...,...,...
10009,88,88_17709_17742,88_152,"fifth , our study focused on highly negative incidents instead of ordinary incidents , which is a typical limitation of critical incident technique ( gremler 2004 ) .",87,limitations and future topics,NaN,PARAGRAPH,1,"Fifth , our study focused on highly negative incidents instead of ordinary incidents , which is a typical limitation of critical incident technique CITE_gremler_d_the_critical_incident_technique_in_service_research_2004_journal_of_service_research .",1.0,Limitation
10010,88,88_17827_17836,88_153,our findings open up possibilities for future research .,87,limitations and future topics,NaN,PARAGRAPH,0,Our findings open up possibilities for future research .,NaN,Future Work
10011,88,88_17836_17858,88_153,"first , our data imply that users also engage in coping efforts beforehand to prevent potential negative incidents in the future .",87,limitations and future topics,NaN,PARAGRAPH,0,"First , our data imply that users also engage in coping efforts beforehand to prevent potential negative incidents in the future .",NaN,Future Work
10012,88,88_17884_17911,88_153,; newby - clark 2004 ) is an interesting area for future studies on information technology use .,87,limitations and future topics,NaN,PARAGRAPH,2,; END_CITE CITE_neupert_s_solving_tomorrows_problems_today_daily_anticipatory_coping_and_reactivity_to_daily_stressors_2016_book CITE_newbyclark_i_getting_ready_for_the_bad_times_selfesteem_and_anticipatory_coping_2004_european_journal_of_social_psychology is an interesting area for future studies on information technology use .,1.0,Future Work


In [200]:
Labelled_data.to_csv(r'C:\Users\nitis\OneDrive\Desktop\thesis\Nitish_datasets\Nitish_datasets\labelled.csv')

TypeError: to_csv() got an unexpected keyword argument 'index_col'

# Relation to literature and defination can be same sometime
Example text : we define digital platformization and infrastructuring and has to control both demand driven innovation as a set of digital resources including services and content that enable value creating interactions between external producers and consumers cf parker et al

Solution : Dont have same string in relation to literature like in Defination

It is same with aim and motivation

Example text :  our aim was to provide an in depth analysis of the field rather than providing a descriptive overview jones and gatrell	


# Relation to literature and hypothesis can be same sometime

we also calculated the statistical power of our model that is the probability of not rejecting an incorrect model diamantopoulos and siguaw	



# Relation to literature and future work can be same sometime

our study accordingly follows a quantitative exploratory approach in which we use a large dataset hey et al which leads to a set of propositions to guide future research on an under researched topic that bears tremendous importance for our understanding of creativity in the digital age	